## *Pré-processamento*

Esta é uma etapa importante do processamento de dados, que envolve vários conceitos. No pré-processamento, os dados podem ser "limpados", padronizados e transformados.

- *Limpar* os dados refere-se à remoção ou correção de dados que estejam incompletos, corrompidos ou imprecisos.
- *Padronizar* os dados refere-se à remoção de *outliers*, além de deixar todas as possíveis mensurações na mesma escala e unidade, além de normalizá-las quando necerrário.
- *Transformar* os dados refere-se à aplicação dos dados em um formato que favoreça futuras extrações de características ou análises.

In [1]:
# Realização de imports necessários

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy.signal import stft

## Obtenção dos dados no domínio do tempo e da frequência

In [2]:
data = np.load('datasets/OpenBCI_GUI-v5-meditation.npy')
print(data.shape[0])
data.shape[0] / 8
data = data.swapaxes(1, 0)

# domínio do tempo
data.shape

175952


(8, 175952)

In [3]:
f, t, Zxx = stft(data, fs=250., nperseg=256)
print(f.shape, t.shape, Zxx.shape)
freqs = Zxx.reshape(8, len(f) * len(t))
freqs.shape

(129,) (1376,) (8, 129, 1376)


(8, 177504)

Tarefa:
- Realizar plots com matplotlib dos dados em ambos os domínios para cada um dos canais
- Fazer a média dos 8 eletrodos e realizar os plot novamente

Desafio:
- Obter o PSD ($mod(w)^2$) para os valores no domínio da frequência e plotar novamente

## Definição das funções para aplicação de filtros temporais

Para a aplicação dos filtros temporais, serão utilizadas as seguintes funções a seguir:

In [4]:
def butter_bandpass(data, lowcut, highcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_lowpass(data, lowcut, fs=200, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)


def butter_highpass(data, highcut, fs=200, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=200, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

## Algumas funções extras para avaliar ...

- Utilizar o `scipy.signal.welch` para obter as Frequências em PSD com o método Welch, e comparar com os dados (no gráfico) com dados de frequências obtidas com `scipy.signal.stft` $\rightarrow$ PSD *handcrafted*
- Imprimir apenas "partes importantes" do sinal no domínio da frequência
- Fazer um "merge" dos resultados obtidos pelos alunos nas tarefas e desafio.

## Desenvolvimento de funções para `plot`

Vamos criar uma função que será utilizada como padrão para imprimir os gráficos no domíno do tempo e da frequência.

**Obs.:** Para imprimir os dados no domínio da frequência, foi utilizado a função `matplotlib.pyplot.psd` que na verdade utiliza a transformação dos dados por FFT, porém aplica uma característica que evidencia a "energia" dos dados, chamado de PSD (_Power Spectrum Density_)

In [5]:
def print_graphs(data):
    for i in range(data.shape[1]):
        plt.plot(data[3,i,:])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(data.shape[1]):
        plt.psd(data[3,i,:], Fs=200)
    plt.title('Domínio da frequência')
    plt.show()